In [51]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from tqdm import tqdm
from scipy.stats import linregress
from utils.preprocessing import get_texts
from utils.preprocessing import get_texts, stop_words
import random
from sklearn.metrics import confusion_matrix 
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /Users/luckywang/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [76]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")
df_esg_score = df_esg_score.dropna()

In [77]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [79]:
len(df_esg_score)

327

In [17]:
training_comp_training = pd.DataFrame(index = ['score'])
# validate_comp_train = pd.DataFrame(index = ['score'])
results_training = pd.DataFrame(index=['accuracy', 'precision', 'recall'])

## Util Functions

In [23]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))

def get_cik(ticker):
    """ Get the cik for the ticker specified by the input argument 
    Input:
        ticker(str): ticker of the company e.g. "FB"
    """
    return ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:]
    
def ngrams(s, n):
    """ Get all the n-gram for input texts s
    Input:
        s (str): A string of texts with each word separated by a whitespace
        n (int): n-gram to extract
    Return:
        [str]: A list of string in the following format ([['a', 'b'], ['b', 'c'], ['c', 'd']])
    """
    
    s = s.split(' ')
    output = []
    for i in range(len(s) - n + 1):
        output.append(s[i:i+n])

    return output

def get_count(doc, df_dict, n_min, n_max):
    """ Count the number of good and bad words occurred in the document
    Input:
        doc (str): A string with all the words in the documents
        df_dict (pd.DataFrame): A DataFrame with word and isGood column, generated by previous section
        n_min, n_max (int): specify the ngram range used to generate the dictionary, should be consistent with how df_dict is generated
    Return:
        (dict): A dictionary with value good_count and bad_count
    """
    # grams = doc.split()

    grams = []
    for n in range(n_min, n_max + 1):
        grams.extend([' '.join(li) for li in ngrams(doc, n)])

    good_count = bad_count = 0
    s = 0
    
    for g in grams:
        if g in df_dict["word"].values:
            val = df_dict[df_dict["word"] == g]["isGood"].values
            if val > 0:
                good_count += 1
            elif val <= 0:
                bad_count += 1
            s += val
    print(s)
    return {"good_count": good_count, "bad_count": bad_count, "score": s}

def normalize(array):
    mean = np.mean(array)
    std = np.std(array)

    return (array - mean) / std

    
def validation(df_topk, val_tickers, score_scheme="binary", alpha=0.3):
    """ Perform the validation step
    The validation rationale: Companies whose score are in upper 50% group are considered "bad" companies and the corresponding val_true = 1; 0 otherwise (in lower 50% group, which is considered a good company)
    Input:
        df_topk (pd.DataFrame): containes the sector specific dict
        val_tickers (list): A list of tickers to be validated
        score_scheme: score_scheme in ("binary", "ratio", "ratio_idf")
    """
    assert score_scheme in ("binary", "ratio", "ratio_idf", "ratio_norm", "ratio_kernel")
    if score_scheme == "binary":
        diff = df_topk["good_nums"] - df_topk["bad_nums"]
        upper_threshold = np.quantile(diff, 1 - alpha)
        lower_threshold = np.quantile(diff, alpha)

        df_topk["isGood"] = diff.apply(lambda x: 1 if x > upper_threshold else (
        -1 if x < lower_threshold else 0))
    elif score_scheme == "ratio":
        # log(good_count / (bad_count + 1))
        # Make sure 80/40 and 40/80 have same magnitude after transformation
        df_topk["isGood"] = np.log2((df_topk["good_nums"] + 1) / (df_topk["bad_nums"] + 1))

    # elif score_scheme == "ratio_idf":
    #     # log(good_count / (bad_count + 1))
    #     # Make sure 80/40 and 40/80 have same magnitude after transformation
    #     total = df_topk["good_nums"].values + df_topk["bad_nums"].values
    #     ma = np.max(total)
    #     df_topk["isGood"] = ((df_topk["good_nums"] + 1) / (df_topk["bad_nums"] + 1)) * np.log(ma / total)

    elif score_scheme == "ratio_norm":
        # log(good_normalization / bad_normalization)
        good_norm = df_topk["good_nums"].values
        bad_norm = df_topk["bad_nums"].values
        df_topk["isGood"] = np.log2((good_norm + 1)/(bad_norm + 1))
    
    elif score_scheme == "ratio_kernel":
        # good_nums / (bad_nums + 1) / total
        good_norm = df_topk["good_nums"].values
        bad_norm = df_topk["bad_nums"].values
        total = df_topk["good_nums"].values + df_topk["bad_nums"].values
        ma = np.max(total)
        weights = 1 - ((total - ma) ** 2 / (ma ** 2))       
        # weight function rationale: Words with high or low occurrence -> not preferrable
        # values in between should have higher weights
        df_topk["isGood"] = np.log2((good_norm + 1)/(bad_norm + 1) * weights)


    print(df_topk)
    # 1 if good_nums - bad_nums > threshold; -1 if good_nums - bad_nums < -threshold; 0 otherwise
    val_ciks = [get_cik(ticker) for ticker in val_tickers]
    
    ret_texts = get_texts(val_ciks, val_tickers) 

    val_pred = []
    for doc in tqdm(ret_texts["docs"]):
        ret = get_count(doc, df_topk[["word", "isGood"]], 2, 3)

        # if ret["good_count"] - ret["bad_count"] > 0:
        #     val_pred.append(1)
        # else:
        #     val_pred.append(0)
        if ret["score"] > 0:
            val_pred.append(1)
        else:
            val_pred.append(0)
    
    print("val_pred: {}".format(val_pred))
    
    return val_pred


/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Model validation
(Bigram and Trigram dictionary)

In [97]:
ciks2ticker = dict()

In [101]:
def get_ciks(tickers):
    ciks = []

    for ticker in tickers:
        try:
            # for a given ticker, find its cik number through th ticker library
            cik = get_cik(ticker)
            ciks.append(cik)
            ciks2ticker[cik] = ticker
        except:
            continue

    return ciks

In [102]:
def get_tickers(ciks):
    return [ciks2ticker[cik] for cik in ciks]

In [86]:
def train_dict_goodvbad(good_ticker, bad_ticker, score_type, sector=None, rerun=False):
    """
    Train dictionary based on method 2: Good versus Bad. Refer to Section 3.4 of the paper
    Input:
        good_ticker (list): A list of good companies' tickers
        bad_ticker (list): A list of bad companies' tickers
        score_type (str): Specify the the score_type
        sector (str): Specify the sector name; If none, then build non sector-specific dictionary
        rerun (bool): If rerun == True, retrain a dictionary even if a dictionary already exists
    Return:
        df (DataFrame): A dictionary with index being words/bigrams and an associated score
    """
    if sector:
        path = os.path.join("data", "goodvbad", "train", "{}_{}.csv".format(sector, score_type))
    else:
        path = os.path.join("data", "goodvbad", "train", "{}.csv".format(score_type))
    
    if not rerun and os.path.exists(path):
        dict_goodvbad = pd.read_csv(path, index_col=0)
    else:
        if sector:
            print("Train {} {}".format(sector, score_type))
        else:
            print("Train {}".format(score_type))

        good_cik = get_ciks(good_ticker)
        bad_cik = get_ciks(bad_ticker)

        good_ticker = get_tickers(good_cik)
        bad_ticker = get_tickers(bad_cik)

        ret_good = get_texts(good_cik, good_ticker)
        ret_bad = get_texts(bad_cik, bad_ticker)

        good_docs = ret_good["docs"]
        bad_docs = ret_bad["docs"]

        n_min = 2
        n_max = 3
        cv = CountVectorizer(max_df=0.7, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
        word_count_vector = cv.fit_transform(good_docs + bad_docs)
        count_feature = word_count_vector.toarray().sum(axis=0)
        feature_names = cv.get_feature_names()

        
        d = {"count": [], "good_nums": [], "bad_nums": [], "isGood": []}

        for feature_idx, word in enumerate(feature_names):
            # good_sum = bad_sum = good_num = bad_num = 0
            good_num = bad_num = 0

            for i, doc_set in enumerate(good_docs):
                if word in doc_set:
                    good_num += 1
            for i, doc_set in enumerate(bad_docs):
                if word in doc_set:
                    bad_num += 1
            
            d["count"].append(count_feature[feature_idx])
            d["good_nums"].append(good_num)
            d["bad_nums"].append(bad_num)

        diff = np.array(d["good_nums"]) - np.array(d["bad_nums"])
        
        alpha = 0.3
        upper_score = np.quantile(diff, 1 - alpha)
        lower_score = np.quantile(diff, alpha)
        d["isGood"] = np.where(diff > upper_score, 1, 0) + np.where(diff < lower_score, -1, 0)

        dict_goodvbad = pd.DataFrame(d, index=feature_names)
        dict_goodvbad.to_csv(path)

    return dict_goodvbad

In [88]:
def train_dict_tfidf(good_ticker, bad_ticker, score_type, sector=None, rerun=False):
    """
    Train dictionary based on method 1: Good versus Bad. Refer to Section 3.3 of the paper
    Input:
        good_ticker (list): A list of good companies' tickers
        bad_ticker (list): A list of bad companies' tickers
        score_type (str): Specify the the score_type
        sector (str): Specify the sector name; If none, then build non sector-specific dictionary
        rerun (bool): If rerun == True, retrain a dictionary even if a dictionary already exists
    Return:
        df (DataFrame): A dictionary with index being words/bigrams and an associated score
    """
    if sector:
        path = os.path.join("data", "tfidf_scores", "train", "{}_{}.csv".format(sector, score_type))
    else:
        path = os.path.join("data", "tfidf_scores", "train", "{}.csv".format(score_type))

    if not rerun and os.path.exists(path):
        df_tfidf = pd.read_csv(path, index_col=0)
    else:
        if sector:
            print("Train {} {}".format(sector, score_type))
        else:
            print("Train {}".format(score_type))
        
        tickers = good_ticker + bad_ticker
        ciks = get_ciks(tickers)
        tickers = get_tickers(ciks)

        esgs = df_esg_score[df_esg_score["Company"].isin(tickers)][["Company", "socialScore", "governanceScore", "environmentScore"]]

        ret = get_texts(ciks, tickers)
        docs = ret["docs"]

        cv = CountVectorizer(max_df=0.8, stop_words=stop_words, max_features=1000)
        word_count_vector = cv.fit_transform(docs)

        tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
        tfidf_transformer.fit(word_count_vector)

        feature_names = cv.get_feature_names()

        df_doc_word = pd.DataFrame(columns=feature_names, index=tickers)

        for i, ticker in tqdm(enumerate(tickers)):
            tf_idf_vector = tfidf_transformer.transform(cv.transform([docs[i]]))
            
            coo_matrix = tf_idf_vector.tocoo()
            # coo_matrix: A sparse matrix in which coo_matrix.col stores word_idx, coo_matrix.data stores tfidf score
            
            tuples = zip(coo_matrix.col, coo_matrix.data)
            for word_idx, tfidf in tuples:
                df_doc_word.at[ticker, feature_names[word_idx]] = tfidf

        df_doc_word = df_doc_word.iloc[:, df_doc_word.columns.isin(words.words())]
        df_doc_word = df_doc_word.fillna(0)

        feature_names = [name for name in feature_names if name in words.words()]

        df_tfidf = pd.DataFrame(columns=["{}_beta".format(score_type)], index=feature_names)

        # for typ in ["social", "governance", "environment"]:
        score = esgs[score_type]
        slopes = []
        
        for word in feature_names:
            tfidfs = df_doc_word[word].values.astype(float)
            slope, intercept, *_ = linregress(tfidfs, score)
            slopes.append(slope)
        df_tfidf["{}_beta".format(score_type)] = slopes

        cols = df_tfidf.columns
        alpha = 0.3

        for col in cols: 
            betas = df_tfidf[col]
            score_type = col.split('_')[0]
            
            upper_score = np.quantile(betas, 1 - alpha)
            lower_score = np.quantile(betas, alpha)
            
            is_good = np.where(betas < lower_score, 1, 0) + np.where(betas > upper_score, -1, 0)
            
            df_tfidf["isGood"] = is_good

        df_tfidf.to_csv(path)
    return df_tfidf

In [89]:
def validation_tfidf(df_tfidf, val_tickers):
    """ Perform the validation step
    The validation rationale: Companies whose scores are in upper group are considered "bad" companies and the corresponding val_true = 1; 0 otherwise (in lower group, which is considered a good company)
    Input:
        df_topk (pd.DataFrame): containes the sector specific dict
        val_tickers (list): A list of tickers to be validated
    Return:
        val_pred (list): A list of validation prediction
    """
    df_tfidf["word"] = df_tfidf.index
    
    # 1 if good_nums - bad_nums > threshold; -1 if good_nums - bad_nums < -threshold; 0 otherwise
    val_ciks = [get_cik(ticker) for ticker in val_tickers]
    
    ret_texts = get_texts(val_ciks, val_tickers)

    val_pred = []
    for i, doc in tqdm(enumerate(ret_texts["docs"])):
        ret = get_count(doc, df_tfidf[["word", "isGood"]])
        
        if ret["good_count"] - ret["bad_count"] > 0:
            val_pred.append(1)
        else:
            val_pred.append(0)
    
    print("val_pred: {}".format(val_pred))
    
    return val_pred

In [90]:
def get_report(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    prec = cm[1][1]/(cm[1][1]+cm[1][0])
    rec = cm[1][1]/(cm[1][1]+cm[0][1])
    acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

    print("precision: \n{}".format(prec))
    print("recall: \n{}".format(rec))
    print("accuracy: \n{}".format(acc))
    print("Confusion Matrix: \n{}".format(cm))

    return {"cm": cm, "precision": prec, "recall": rec, "accuracy": acc}


In [91]:
sectors = ["Consumer Cyclical"]     #['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare', 'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology', 'Financial Services', 'Communication Services', 'Real Estate']
score_types = ["governanceScore", "environmentScore", "socialScore"]       #["governanceScore", "environmentScore", "socialScore"]

In [92]:
df_val_precision = pd.DataFrame(index=sectors, columns=score_types)
df_val_recall = pd.DataFrame(index=sectors, columns=score_types)
df_val_accuracy = pd.DataFrame(index=sectors, columns=score_types)
cms = []

In [93]:
approach = "goodvbad" # ("tfidf", "goodvbad")

In [94]:
all_dicts = []

In [95]:
training_results = pd.DataFrame(index=['accuracy', 'precision', 'recall', 'training_set'])
validation_results = pd.DataFrame(index=['accuracy', 'precision', 'recall', 'training_set', 'validation_set'])

### 1. Validatation of dictionary based on score types

In [ ]:
for score_type in score_types:
    print("{}".format(score_type))
    esgs = df_esg_score[["Company", "socialScore", "governanceScore", "environmentScore"]]
    score = esgs[score_type]
    alpha = 0.3
    upper_score = np.quantile(score, 1 - alpha)
    lower_score = np.quantile(score, alpha)

    good_companies = list(esgs[esgs[score_type] < lower_score]["Company"].values)
    bad_companies = list(esgs[esgs[score_type] > upper_score]["Company"].values)
            
    #training set
    train_good = random.sample(list(good_companies), int(len(good_companies) * 0.7))
    train_bad = random.sample(list(bad_companies), int(len(bad_companies) * 0.7))

    if approach == "goodvbad":
        df_dict = train_dict_goodvbad(train_good, train_bad, score_type, sector=None, rerun=False)
        
        if "word" not in df_dict.columns:
            df_dict["word"] = df_dict.index.to_numpy()

    if approach == "tfidf":
        df_dict = train_dict_tfidf(train_good, train_bad, score_type, sector=None, rerun=False)
    
    all_dicts.append(df_dict)

    #validation set
    validate_good = [ticker for ticker in good_companies if ticker not in train_good]
    validate_bad = [ticker for ticker in bad_companies if ticker not in train_bad]

    val_tickers = validate_good + validate_bad
    
    # val_pred = validation_tfidf(df_dict, val_tickers)   # df is the dictionary with good_num, bad_num, diff
    val_pred = validation(df_dict, val_tickers, score_scheme="ratio_idf")
    val_true = [1] * len(validate_good) + [0] * len(validate_bad)

    val_performance = get_report(val_true, val_pred)

    cm = val_performance["cm"]
    prec = cm[1][1]/(cm[1][1]+cm[0][1])
    rec = cm[1][1]/(cm[1][1]+cm[1][0])
    tpr = cm[1][1]/(cm[1][1]+cm[0][1])
    tnr = cm[0][0]/(cm[0][0]+cm[1][0])
    acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

    num_train = len(list(train_good) + list(train_bad))
    num_validate = len(validate_good + validate_bad)

    training_results.at['precision', score_type] = prec
    training_results.at['recall', score_type] = rec
    training_results.at['accuracy', score_type] = acc
    training_results.at['training_set', score_type] = num_train
        

In [ ]:
training_results

In [ ]:
df_dict

In [40]:
df_dict["isGood"].value_counts()

 0    129
 1     56
-1     15
Name: isGood, dtype: int64

### 2. Validatation of dictionary based on full ESG scores

In [104]:
esgs

,Company,socialScore,governanceScore,environmentScore
0,LEG,17.19,11.23,20.38
1,COG,14.01,9.28,23.39
2,GE,15.72,11.98,15.65
3,MRO,10.27,8.70,23.76
4,CVX,10.67,10.21,20.29
...,...,...,...,...
442,HAS,5.33,5.07,0.05
444,AVB,2.92,4.40,2.95
445,PLD,3.38,4.23,2.44
446,RHI,5.76,3.56,0.07


In [116]:
score = df_esg_score["totalEsg"]
alpha = 0.3
upper_score = np.quantile(score, 1 - alpha)
lower_score = np.quantile(score, alpha)

good_companies = list(esgs[esgs[score_type] < lower_score]["Company"].values)
bad_companies = list(esgs[esgs[score_type] > upper_score]["Company"].values)
        
#training set
train_good = random.sample(list(good_companies), int(len(good_companies) * 0.7))
train_bad = random.sample(list(bad_companies), int(len(bad_companies) * 0.7))

if approach == "goodvbad":
    df_dict = train_dict_goodvbad(train_good, train_bad, score_type, sector=None, rerun=True)
    
    if "word" not in df_dict.columns:
        df_dict["word"] = df_dict.index.to_numpy()

if approach == "tfidf":
    df_dict = train_dict_tfidf(train_good, train_bad, score_type, sector=None, rerun=True)

all_dicts.append(df_dict)

#validation set
validate_good = [ticker for ticker in good_companies if ticker not in train_good]
validate_bad = [ticker for ticker in bad_companies if ticker not in train_bad]

val_tickers = validate_good + validate_bad

# val_pred = validation_tfidf(df_dict, val_tickers)   # df is the dictionary with good_num, bad_num, diff
val_pred = validation(df_dict, val_tickers, score_scheme="ratio_idf")
val_true = [1] * len(validate_good) + [0] * len(validate_bad)

val_performance = get_report(val_true, val_pred)

cm = val_performance["cm"]
prec = cm[1][1]/(cm[1][1]+cm[0][1])
rec = cm[1][1]/(cm[1][1]+cm[1][0])
tpr = cm[1][1]/(cm[1][1]+cm[0][1])
tnr = cm[0][0]/(cm[0][0]+cm[1][0])
acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

num_train = len(list(train_good) + list(train_bad))
num_validate = len(validate_good + validate_bad)

training_results.at['precision', 'ESG'] = prec
training_results.at['recall', 'ESG'] = rec
training_results.at['accuracy', 'ESG'] = acc
training_results.at['training_set', 'ESG'] = num_train

Train governanceScore


4it [00:00,  5.33it/s]

Scraping CIK 0000100493


100%|██████████| 1/1 [00:05<00:00,  5.93s/it]


Scraping CIK 0000100493


...

[911]


 80%|████████  | 4/5 [01:28<00:30, 30.02s/it]

[19351]


100%|██████████| 5/5 [02:32<00:00, 30.57s/it]

[20929]
val_pred: [0, 1, 1, 1, 1]
precision: 
0.8
recall: 
1.0
accuracy: 
0.8
Confusion Matrix: 
[[0 0]
 [1 4]]


### 3. Validatation of sector and score type specific dictionary

In [118]:
precision = pd.DataFrame()
recall = pd.DataFrame()
accuracy = pd.DataFrame()
true_pos = pd.DataFrame()
true_neg = pd.DataFrame()
training_comp = pd.DataFrame()
validate_comp = pd.DataFrame()

In [ ]:
for sector in sectors:
    for score_type in score_types:
        print("{} {}".format(sector, score_type))
        esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]
        score = esgs[score_type]
        alpha = 0.3
        upper_score = np.quantile(score, 1 - alpha)
        lower_score = np.quantile(score, alpha)

        good_companies = list(esgs[esgs[score_type] < lower_score]["Company"].values)[:5]
        bad_companies = list(esgs[esgs[score_type] > upper_score]["Company"].values)[:5]
                
        #training set
        train_good = random.sample(list(good_companies), int(len(good_companies) * 0.7))
        train_bad = random.sample(list(bad_companies), int(len(bad_companies) * 0.7))

        if approach == "goodvbad":
            df_dict = train_dict_goodvbad(train_good, train_bad, score_type, sector=None, rerun=True)
            
            if "word" not in df_dict.columns:
                df_dict["word"] = df_dict.index.to_numpy()

        if approach == "tfidf":
            df_dict = train_dict_tfidf(train_good, train_bad, score_type, sector=None, rerun=True)
        
        all_dicts.append(df_dict)

        #validation set
        validate_good = [ticker for ticker in good_companies if ticker not in train_good]
        validate_bad = [ticker for ticker in bad_companies if ticker not in train_bad]

        val_tickers = validate_good + validate_bad
        
        # val_pred = validation_tfidf(df_dict, val_tickers)   # df is the dictionary with good_num, bad_num, diff
        val_pred = validation(df_dict, val_tickers, score_scheme="ratio")
        val_true = [1] * len(validate_good) + [0] * len(validate_bad)

        val_performance = get_report(val_true, val_pred)

        cm = val_performance["cm"]
        prec = cm[1][1]/(cm[1][1]+cm[0][1])
        rec = cm[1][1]/(cm[1][1]+cm[1][0])
        tpr = cm[1][1]/(cm[1][1]+cm[0][1])
        tnr = cm[0][0]/(cm[0][0]+cm[1][0])
        acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

        num_train = len(list(train_good) + list(train_bad))
        num_validate = len(validate_good + validate_bad)

        training_comp.at[sector, score_type] = num_train
        validate_comp.at[sector, score_type] = num_validate
        
        precision.at[sector, score_type] = prec
        recall.at[sector, score_type] = rec
        accuracy.at[sector, score_type] = acc
        true_pos.at[sector, score_type] = tpr
        true_neg.at[sector, score_type] = tnr
        accuracy.at[sector, score_type] = acc
            